In [ ]:
import cv2

from matplotlib import pyplot as plt

In [ ]:
mmap0 = cv2.imread("../inputs/mmap0.png")
mmap1 = cv2.imread("../inputs/mmap1.png")

In [ ]:
fusiooon = cv2.addWeighted(mmap0, 0.5, mmap1, 0.5, 0)

In [ ]:
plt.imshow(fusiooon)

In [ ]:
def enhance_fg(img):
    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(gimg,110,255,cv2.THRESH_BINARY)
    
    boxed = cv2.boxFilter(thresh1, -1, (40, 40), normalize=False)
    alpha = cv2.blur(boxed, (11, 11))
    alpha = alpha.reshape(*alpha.shape, 1).astype('float32')/255

    fg = (img*alpha).astype('uint8')
    composed = cv2.addWeighted(img, 0.2, fg, 0.8, 0)
    return composed

In [ ]:
mmap0 = enhance_fg(mmap0)
mmap1 = enhance_fg(mmap1)

In [ ]:
cross = cv2.imread("../inputs/cross.png")
plt.imshow(cross)

In [ ]:
detector = cv2.AgastFeatureDetector.create(threshold=40)
detector = cv2.KAZE.create()
descriptor = cv2.SIFT_create()

kps = detector.detect(cross, None)
len(kps)

kps, feat = descriptor.compute(cross, kps)

In [ ]:
img2 = cv2.drawKeypoints(cross, kps[1:2], None, color=(0,255,0), flags=0)
plt.imshow(img2), plt.show()

In [ ]:
plt.imshow(mmap0)

In [ ]:
brisk = cv2.BRISK.create(thresh=100)

agast = cv2.AgastFeatureDetector.create()

img = mmap0.copy()

kp = agast.detect(img,None)
# compute the descriptors with ORB

center = np.float32(mmap0.shape[1::-1]) / 2
kp = [pt for pt in kp if np.linalg.norm(pt.pt - center) < 115]

kp, des = brisk.compute(img, kp)
# draw only keypoints location,not size and orientation
img2 = cv2.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
plt.imshow(img2), plt.show()

In [ ]:
from helpers import *

In [ ]:
def kp_description(frame, filter_radius=False, use_binary=False):
    descriptor = cv2.KAZE.create() if use_binary else cv2.KAZE.create(upright=True)#cv2.SIFT_create()

    detector = cv2.KAZE.create(upright=True)

    kp1 = detector.detect(frame)
    
    if filter_radius:
        kp1 = filter_kp(kp1, frame)
    
    kp1, des1 = descriptor.compute(frame, kp1)

    return kp1, des1

def kp_matching(des1, des2, ann=False, k=2, use_binary=False):
    if ann: #WARNING Not finished
        FLANN_INDEX_LSH = 6
        index_params= dict(algorithm = FLANN_INDEX_LSH,
                       table_number = 6, # 12
                       key_size = 12,     # 20
                       multi_probe_level = 1) #2
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(descriptors0, descriptors1, k=k)
    else: #slower but not approximate
        bf = cv2.BFMatcher(cv2.NORM_HAMMING if use_binary else cv2.NORM_L2)
        matches = bf.knnMatch(des1,des2,k=k)
    return matches

In [ ]:
def filter_kp(kp, img, radius=115):
    center = np.float32(img.shape[1::-1]) / 2
    return [pt for pt in kp if np.linalg.norm(pt.pt - center) < 115]

def binary_kp_description(frame1, frame2, filter_radius=True, use_orb=False):
    algo = cv2.ORB_create() if use_orb else cv2.BRISK.create(thresh=50)
    algo = cv2.SIFT_create()
    detector = cv2.KAZE.create(upright=True)
    
    kp1 = detector.detect(frame1,None)
    kp2 = detector.detect(frame2,None)
    
    if filter_radius:
        kp1 = filter_kp(kp1, frame1)
        kp2 = filter_kp(kp2, frame2)
    
    kp1, des1 = algo.compute(frame1, kp1)
    kp2, des2 = algo.compute(frame2, kp2)

    return kp1, des1, kp2, des2

def binary_kp_matching(des1, des2, ann=False, k=2):
    if ann: #WARNING NEEDS testing
        FLANN_INDEX_LSH = 6
        index_params= dict(algorithm = FLANN_INDEX_LSH,
                       table_number = 6, # 12
                       key_size = 12,     # 20
                       multi_probe_level = 1) #2
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(descriptors0, descriptors1, k=k)
    else: #slower but not approximate
        bf = cv2.BFMatcher(cv2.NORM_HAMMING)
        matches = bf.knnMatch(des1,des2,k=k)
    return matches

def filter_matches(matches, ratio_test=False):
    if ratio_test:
        good = []
        for m, n in matches: #Lowe's SIFT value may not be relevant for binary descriptors
            if m.distance < 0.9 * n.distance:
                good.append(m)
        return good
    else:
        return [m[0] for m in matches]
    
def draw_matches(frame1, kp1, frame2, kp2, good, matchesMask=None):
    draw_params = dict(matchColor=(0, 255, 0),  # draw matches in green color
                       singlePointColor=None,
                       matchesMask=matchesMask,  # draw only inliers
                       flags=2)

    comparison = cv2.drawMatches(frame1, kp1, frame2, kp2, good, None, **draw_params)
    return comparison

In [ ]:
use_binary = False

cross = cv2.imread("../inputs/cross.png")

cross_kp, cross_des = kp_description(cross, use_binary=use_binary)

kp1, des1 = kp_description(mmap0, use_binary=use_binary)
kp2, des2 = kp_description(mmap1, use_binary=use_binary)

In [ ]:
img2 = cv2.drawKeypoints(mmap0, kp1, None, color=(0,255,0), flags=0)
plt.imshow(img2)

In [ ]:
def filter_with_cross(cross_des, kp1, des1):

    matches = kp_matching(cross_des, des1, k=20, use_binary=use_binary)
    kp = [kp1[m.trainIdx] for m in matches[0]]
    des = [des1[m.trainIdx] for m in matches[0]]
    return kp, np.vstack(des)  #can do better than 0

kp1, des1 = filter_with_cross(cross_des, kp1, des1)
kp2, des2 = filter_with_cross(cross_des, kp2, des2)

In [ ]:
img2 = cv2.drawKeypoints(mmap0, kp1, None, color=(0,255,0), flags=0)
plt.imshow(img2)

In [ ]:
img2 = cv2.drawKeypoints(mmap1, kp2, None, color=(0,255,0), flags=0)
plt.imshow(img2)

In [ ]:
matches = kp_matching(des1, des2, k=2, use_binary=use_binary)
matches = filter_matches(matches, ratio_test=False)

In [ ]:
comp = draw_matches(mmap0, kp1, mmap1, kp2, matches)

In [ ]:
plt.imshow(comp)

In [ ]:
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
m, _ = cv2.estimateAffinePartial2D(src_pts, dst_pts, method=cv2.LMEDS)

In [ ]:
forward = cv2.warpAffine(mmap0, m, mmap1.shape[1::-1])
plt.imshow(cv2.addWeighted(mmap1, 0.5, forward, 0.5, 0))

In [ ]:
backward = cv2.warpAffine(mmap1, m, mmap0.shape[1::-1], flags=cv2.WARP_INVERSE_MAP)
plt.imshow(cv2.addWeighted(mmap0, 0.5, backward, 0.5, 0))

In [ ]:
Mp = np.zeros([3,3])
Mp[:2, :3] = m
Mp[2, 2] = 1

In [ ]:
out = mmap1.copy()

fw_pts = cv2.perspectiveTransform(src_pts, Mp)

for p1, p2 in zip(fw_pts.reshape(-1, 2), dst_pts.reshape(-1, 2)):
    cv2.circle(out, tuple(p1.astype('int')), 3, (0, 0, 255))
    cv2.circle(out, tuple(p2.astype('int')), 4, (255, 0, 0))

In [ ]:
plt.imshow(out)

In [ ]:
loc = np.float32([0, 0]).reshape(-1, 1, 2)

In [ ]:
center = np.float32(mmap0.shape[1::-1]) / 2

In [ ]:
new_center = cv2.perspectiveTransform(center.reshape(-1, 1, 2), Mp)

In [ ]:
delta = new_center - center

In [ ]:
center + delta

In [ ]:
new_center

In [ ]:
center

In [ ]:
loc = np.float32([0, 0])

In [ ]:
delta.ravel()

In [ ]:
loc + delta.ravel()

In [ ]:
!pip install scipy

In [ ]:
(540, 960, 3) (301, 790, 3) (480, 640, 3)

In [ ]:
s = 960 / (790+640)

In [ ]:
s*480, s*640

In [ ]:
s*301, s*790

In [ ]:
render = np.zeros([540+322,960,3])
render[:540,:960,:] = ref
render[540:742,:530] = cv2.resize(comparison, (202,530))
render[540:,530:] = cv2.resize(img, (322,430))